## Importing the required libraries

In [48]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from torch.nn.functional import cosine_similarity
import numpy

### ResNet Feature Extractor

1. **Objective**: Implements a feature extractor using a pretrained `ResNet50` model.
2. **Pretrained Model**: Leverages `ResNet50` with weights pre-trained on the ImageNet dataset for transfer learning.
3. **Feature Extraction**: Excludes the final classification layer by using all layers except the last (`[:-1]`).
4. **Output Format**: The extracted feature map is reshaped to a 2D tensor with dimensions `(batch_size, -1)` for downstream tasks.
5. **Applications**: Useful for tasks like image similarity, clustering, and as input to additional layers or models.
6. **Key Advantage**: Simplifies the extraction of high-dimensional image features from a robust pretrained architecture.


In [15]:
#load the pretrained ResNet Model

class ResNetFeatureExtractior(nn.Module):
    def __init__(self):
        super(ResNetFeatureExtractior,self).__init__()
        resnet = resnet50(pretrained = True)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
    def forward(self,x):
        x = self.feature_extractor(x)
        return x.view(x.size(0),-1)
    

In [ ]:
model = ResNetFeatureExtractior()
model.eval()

### ResNet Feature Extraction with Additional Hidden Layer

1. **Objective**: Enhances the `ResNet50` model by adding a hidden layer, ReLU activation, and dropout to refine feature embeddings.
2. **Feature Extractor**: Utilizes the pretrained `ResNet50` architecture, excluding the final classification layer.
3. **Additional Layer**:
   - Linear layer: Maps 2048-dimensional features to a custom embedding dimension (`embedding_dim`).
   - ReLU: Adds non-linearity for better feature representation.
   - Dropout: Prevents overfitting by randomly dropping 50% of neurons during training.
4. **Embedding Generation**: The model outputs a reduced-dimension feature embedding for each input image.
5. **Customization**: Embedding dimension (`embedding_dim`) can be adjusted based on application requirements.
6. **Key Use Case**: Ideal for applications like image similarity, retrieval, and clustering with refined embeddings.


In [37]:
#load the pretrained ResNet Model with one hideen layer and ReLU layer with random dropout

class ResNetFeatureExtractionWithHiddenLayer(nn.Module):
    def __init__(self,embedding_dim = 512):
        super(ResNetFeatureExtractionWithHiddenLayer,self).__init__()
        resnet = resnet50(pretrained = True)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
        self.additional_layer = nn.Sequential(
            nn.Linear(2048,embedding_dim),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
    def forward(self,x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0),-1)
        x = self.additional_layer(x)
        return x

In [ ]:
HiddenLayerModel = ResNetFeatureExtractionWithHiddenLayer()
HiddenLayerModel.eval()

### Image Transformation Pipeline

1. **Objective**: Prepares input images for model training or inference using a series of transformations.
2. **Resizing**: Rescales the input image to a fixed size of `1024x1024` pixels.
3. **Color Jitter**: Applies random variations in brightness, contrast, saturation, and hue to augment the dataset and improve model generalization.
   - Brightness: Adjusted by up to 20%.
   - Contrast: Adjusted by up to 20%.
   - Saturation: Adjusted by up to 20%.
   - Hue: Adjusted by up to 10%.
4. **Normalization**: Standardizes image pixel values using the ImageNet dataset's mean (`[0.485, 0.456, 0.406]`) and standard deviation (`[0.229, 0.224, 0.225]`).
5. **ToTensor**: Converts the image to a PyTorch tensor, scaling pixel values to the range `[0, 1]`.
6. **Purpose**: Enhances data diversity and ensures input consistency for pretrained models like ResNet.


In [60]:
transform = transforms.Compose([
    transforms.Resize((1024, 1024)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

- **Functionality**: The `load_image` function loads an image from the specified path, converts it to RGB format, applies the defined transformations, and adds a batch dimension.
- **Key Operation**: It ensures compatibility with PyTorch models by transforming and batching the image.


In [40]:
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)

- **Functionality**: This script defines a function `list_jpg_files_in_folder` to recursively list all `.jpg` files in a given folder, returning their names and full paths as a dictionary.
- **Edge Case Handling**: The function checks if the folder exists and handles non-existent paths gracefully.


In [41]:
import os

def list_jpg_files_in_folder(folder_path):
    jpg_file_path_list = []
    if not os.path.exists(folder_path):
        print(f"The folder does not exists : {folder_path}")
        return jpg_file_path_list
    
    for root,_,files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".jpg"):
                full_path = os.path.join(root,file)
                jpg_file_path_list.append({'name':file,'path':full_path})
    return jpg_file_path_list

In [55]:
def get_img_tensor(file_path,model):
    img = load_image(file_path)
    with torch.no_grad():
        embed = model(img)
    return embed

### Function: `get_similarity`

1. **Objective**: Calculates and displays pairwise cosine similarity between image embeddings in a specified folder.
2. **Inputs**:
   - `folder_path`: Path to the folder containing `.jpg` images.
   - `model`: The feature extraction model for generating image embeddings.
3. **Process**:
   - **Step 1**: Use `list_jpg_files_in_folder` to retrieve `.jpg` file paths in the folder.
   - **Step 2**: For each image, generate embeddings using the `get_img_tensor` function.
   - **Step 3**: Normalize embeddings with L2 normalization using `F.normalize`.
   - **Step 4**: Compute cosine similarity between all pairs of embeddings.
   - **Step 5**: Print similarity scores with image file paths.
4. **Key Features**:
   - **Normalization**: Ensures embeddings have unit length, making cosine similarity accurate.
   - **Cosine Similarity**: Measures the angle between embeddings to determine similarity.
5. **Output**: Displays similarity scores between all image pairs in the folder.
6. **Use Case**: Ideal for tasks like identifying similar images in a dataset or verifying image uniqueness.


In [62]:
import torch.nn.functional as F
def get_similarity(folder_path,model):
    img_embed = []
    # folder_path = "./ImgFolderDataset"
    jpg_files = list_jpg_files_in_folder(folder_path)
    for jpg_file in jpg_files:
        filepath = jpg_file.get('path')
        file_embeding = get_img_tensor(filepath,model)
        img_embed.append({'filepath':filepath,'embed':file_embeding})
    for img in img_embed:
        emb1 = F.normalize(img.get('embed'),p=2,dim=1)
        for myimg in img_embed:
            emb2 = F.normalize(myimg.get('embed'),p=2,dim=1)
            similarity = cosine_similarity(emb1,emb2).item()
            print(f"the similarity between {img.get('filepath')} and {myimg.get('filepath')} is {similarity:.4f}")
        

In [65]:
folder_path1 = './animal_image'
get_similarity(folder_path1,model)

the similarity between ./animal_image\Cat\brad1.jpg and ./animal_image\Cat\brad1.jpg is 1.0000
the similarity between ./animal_image\Cat\brad1.jpg and ./animal_image\Cat\cat1.jpg is 0.7756
the similarity between ./animal_image\Cat\brad1.jpg and ./animal_image\Cat\cat2.jpg is 0.7650
the similarity between ./animal_image\Cat\brad1.jpg and ./animal_image\Dog\dog1.jpg is 0.6842
the similarity between ./animal_image\Cat\brad1.jpg and ./animal_image\Dog\dog2.jpg is 0.7085
the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Cat\brad1.jpg is 0.7756
the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Cat\cat1.jpg is 1.0000
the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Cat\cat2.jpg is 0.8265
the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Dog\dog1.jpg is 0.6782
the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Dog\dog2.jpg is 0.6946
the similarity between ./animal_image\Cat\cat2.jpg and ./animal

In [47]:
get_similarity(folder_path1,HiddenLayerModel)


the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Cat\cat1.jpg is 1.0000
the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Cat\cat2.jpg is 0.8801
the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Dog\dog1.jpg is 0.7674
the similarity between ./animal_image\Cat\cat1.jpg and ./animal_image\Dog\dog2.jpg is 0.7416
the similarity between ./animal_image\Cat\cat2.jpg and ./animal_image\Cat\cat1.jpg is 0.8801
the similarity between ./animal_image\Cat\cat2.jpg and ./animal_image\Cat\cat2.jpg is 1.0000
the similarity between ./animal_image\Cat\cat2.jpg and ./animal_image\Dog\dog1.jpg is 0.6783
the similarity between ./animal_image\Cat\cat2.jpg and ./animal_image\Dog\dog2.jpg is 0.6738
the similarity between ./animal_image\Dog\dog1.jpg and ./animal_image\Cat\cat1.jpg is 0.7674
the similarity between ./animal_image\Dog\dog1.jpg and ./animal_image\Cat\cat2.jpg is 0.6783
the similarity between ./animal_image\Dog\dog1.jpg and ./animal_image\